# 식수인원 예측 모델 CNN Manual_Ver3 

---------------------------

### 1. 라이브러리 추가

In [0]:
from __future__ import print_function

import glob
import math
import os

from IPython import display

import pandas as pd
import numpy  as np
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn as sk
from sklearn import metrics
import numpy as np
from sklearn.model_selection import train_test_split

> Tensorflow를 비롯한 다른 라이브러리를 불러옵니다. 데이터 타입과 관련된 warning인데, 무시해도 됩니다. 

----------------------------

### 2. 메뉴판을 만들기 위한 데이터를 불러옵니다.

In [0]:
raw_data = pd.read_csv('ContactMe', encoding = "euc-kr", engine='python') #dtype =np.float64, , keep_default_na=False
raw_data = raw_data.fillna(0)
raw_data.head(5)

In [0]:
menu = raw_data['메뉴']
menu.head(5)

> 제대로 불러왔는지 확인해봅니다

In [0]:
ingre = raw_data[['곡류','두류','난류','묵류','채소류','해조류','어패류','육류','떡류','양념.및.장류','김치류','만두류','면류','과일류']]
ingre.head(1)

,곡류,두류,난류,묵류,채소류,해조류,어패류,육류,떡류,양념.및.장류,김치류,만두류,면류,과일류
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


> ingre 변수에 '재료'와 관련된 변수를 설정합니다. 이는 향후 메뉴판 행렬을 만들 때 사용할 것입니다

In [0]:
method= raw_data[['밥류','죽류','덮밥국밥류','비빔밥볶음밥류','김.초.밥류','국수류','국탕류','찌개류','구이류','찜류','튀김류','무침류','볶음류','장아찌류','전류','조림류','샐러드류','단품류','유제품','빵과자류','음료.및.주류']]
method.head(1)

,밥류,죽류,덮밥국밥류,비빔밥볶음밥류,김.초.밥류,국수류,국탕류,찌개류,구이류,찜류,...,무침류,볶음류,장아찌류,전류,조림류,샐러드류,단품류,유제품,빵과자류,음료.및.주류
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


> method 변수에 '조리방법'과 관련된 변수를 설정합니다. 이 역시 향후 메뉴판 행렬을 만들 때 사용할 것입니다

In [0]:
method = method.values
ingre = ingre.values
ingre_size = ingre.shape[1]
method_size = method.shape[1]

> method 변수에 앞서 설정한 method 변수의 '값'을 저장합니다. ingre 역시 마찬가지로 처리해줍니다. 또한, ingre_size와 method_size에 재료와 조리방식의 크기를 설정해줍니다

2-1. Update 정규화 변수 추가

In [0]:
raw_data2 = pd.read_csv('메뉴별 횟수 및 인원 평균_정규화', encoding = "euc-kr", engine='python') #dtype =np.float64, , keep_default_na=False
raw_data2 = raw_data2.fillna(0)
raw_data2.head(10)

In [0]:
norm1 = raw_data2[['메뉴','normalized']]
norm1 = norm1.values
norm1[0][1]

0.580454545

In [0]:
norm1[0][0]

'가래떡'

In [0]:
norm1.shape[0]

1173

-------------------------------

### 3. 음식 이미지(행렬)를 만들기

In [0]:
ingre.shape[0]

1163

> 메뉴의 전체 데이터 크기를 확인합니다. 즉 전처리할 메뉴-음식이 1163개 있다는 것을 알 수 있습니다

In [0]:
ma = np.zeros((ingre.shape[0],ingre_size,method_size))
ma.shape

(1163, 14, 21)

> 메뉴판을 만들기에 앞서, 먼저 음식 하나하나의 이미지(행렬)를 만들기 위해 0으로 구성되어 있는 행렬을 만듭니다. 

In [0]:
num1 = 0
num2 = 0
num3 = 0
for k in range(ingre.shape[0]):
    num1 = 0
    for i in ingre[k]:
        print(i)
        num2 = 0
        for j in method[k]:
            ma[k][num1][num2] = i*j
            print("ma[", num3, "][", num1, "][", num2, "] = ", ma[k][num1][num2])
            num2 = num2 +1
        num1 = num1 +1
    num3 = num3 +1

> 앞서 만들었던 0으로 된 행렬 ma 행렬에 재료 * 조리방식을 넣어줍니다. 행렬 각각의 칸에 0 아니면 1이 들어갈 것입니다.

In [0]:
a = dict()
for k in range(ingre.shape[0]):
    a[menu[k]] = ma[k]
    print('menu : ', menu[k])
    print('array : ', a[menu[k]])

---보류 / Update

In [0]:
for k in range(ingre.shape[0]):
    for i in range(norm1.shape[0]):
        if menu[k] == norm1[i][0] :
            a[menu[k]] = a[menu[k]] * norm1[i][1]
            print('Update Menu: ')
            print(menu[k])
            print('Update Norm: ')
            print(a[menu[k]])


------------------------

> 각 음식 이미지(행렬)이 만들어진 것을 확인해봅니다

--------------------------

### 4. 날짜별로 메뉴판 만들기

In [0]:
raw_data = pd.read_csv('dummy', encoding = "CP949", engine='python') #dtype =np.float64, , keep_default_na=False
raw_data.head(5)

> date와 l.men이 담긴 데이터를 불러옵니다.

In [0]:
b1 = raw_data[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12']]
b2 = raw_data[['date', 'l.men']]
b1.columns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
b1.head(4)

> 열 값을 V1~V12가 아닌 0~11까지의 값으로 바꿔줍니다

In [0]:
ma = np.zeros((b1.shape[0],ingre_size,method_size))
ma.shape

(741, 14, 21)

> 앞서 음식 이미지(행렬)을 만든 것과 마찬가지로 0으로 만들어진 행렬을 만듭니다. 여기에는 날짜별 메뉴판이 담길 것입니다

In [0]:
for i in range(b1.shape[0]):
    print('date : ', b2['date'][i])
    print('l.men : ', b2['l.men'][i])
    cnt = 0 
    for k in range(b1.shape[1]): 
        tmp = b1[[0,1,2,3,4,5,6,7,8,9,10,11]][i:i+1]
        if tmp[k].values[0] in a:
            ma[i] += a[tmp[k].values[0]]
            cnt += 1
            print('all_count : ', i+1)
            print('Updating..... ', tmp[k].values[0])
            print('count : ', cnt)
            print('menu : \n', tmp)
            print('array : \n', ma[i])
            print('.')

> 마지막 전처리입니다. 앞서 만든 0 행렬에 날짜별 메뉴를 비교하여 해당하는 (위에서 만든)이미지를 넣어줍니다.
여기서 date는 날짜이며, l.men은 식수인원, all_count는 전체 741개의 날짜 중 몇번째 날짜를 진행하고 있는지를 보여주며, Updating은 어떤 음식을 추가하고 있는지, count는 해당 날짜의 몇번째 음식을 업데이트 하고 있는지 보여줍니다. menu는 해당 날짜의 메뉴를 보여줍니다 

-----------------------

### 5. CNN 학습하기

In [0]:
y = b2['l.men']

In [0]:
trainX, testX, trainY, testY = train_test_split(ma, y, test_size=0.3, random_state=5)

> CNN은 Supervised learning, 즉 지도학습이므로 y(결과)값을 함께 학습해야 합니다. Training과 Test set의 비율은 7:3으로 해줍니다.

In [0]:
def build_model(keep_prob):
  model = keras.Sequential([
    keras.layers.Conv2D(16, kernel_size=(2, 2), activation='relu', padding ='SAME',
                        strides=1, input_shape=[ingre_size,method_size , 1]),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(keep_prob),
      
      
    keras.layers.Conv2D(32, kernel_size=(2, 2), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(keep_prob),
      
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(keep_prob),
    keras.layers.Dense(1, activation='linear')
  ])

  optimizer = tf.train.RMSPropOptimizer(0.01)
  #optimizer=keras.optimizers.Adadelta()
  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model
#model = build_model(drp = 0.5)
build_model(keep_prob = 0.5).summary()
#model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 14, 21, 16)        80        
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 7, 10, 16)         0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 7, 10, 16)         64        
_________________________________________________________________
dropout_42 (Dropout)         (None, 7, 10, 16)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 6, 9, 32)          2080      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 3, 4, 32)          0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 3, 4, 32)          128       
__________

> 모델은 Convolution Layer, MaxPooling Layer, Dropout을 2번에 걸쳐서 사용하였습니다. 즉, 크게 보면 Input과 Output 사이에 2개의 층이 있다고 볼 수 있습니다. 모델을 구성하는 각각의 값들은 10~256까지 여러번 돌린 결과를 보고 가장 나은 결과를 판단하여 구성하였습니다. 즉, 경험에 의존하여 휴리스틱하게 진행하였습니다. 아직 더 튜닝이 필요한 부분입니다. Conv2D와 MaxPooling, Dropout, Relu은 구글링해보시면 쉽게 아실 수 있습니다.

In [0]:
EPOCHS = 10

> EPOCHS는 몇번 iteration 할 것인지를 나타냅니다. 즉 '전체 데이터를 몇번 돌릴 것이냐'에 대한 답이라고 보시면 됩니다. 2만번을 돌린적도, 2천번을 돌린 적도 있지만 천번을 돌렸을 때와 결과가 다르지 않았으므로, Minimum Point는 천번이라고 판단하여 천번만 돌리도록 하겠습니다.

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
sess.run(build_model(0.9))

In [0]:
# Store training stats
history = build_model(keep_prob = 0.5).fit(trainX.reshape([-1,ingre_size, method_size,1]), trainY, 
                    epochs=EPOCHS
                    )

> 모델을 학습시킴과 동시에 그 값을 history에 저장하였습니다. MAE(Mean Absolute Error)값이 800에서 185 정도로 수렴하는 것을 볼 수 있지만 미세하게 값이 왔다갔다 하는 것을 보아 좀더 튜닝이 필요한 것을 볼 수 있습니다.

-----------------------------

### 6. 모델 평가하기

In [0]:
loss_and_metrics = build_model(keep_prob = 0.9).evaluate(testX.reshape([-1,ingre_size, method_size,1]), testY)
print('loss_and_metrics : ' + str(loss_and_metrics))

> 앞서 나눈 Test 값을 통해 모델을 평가합니다. MAE 값이 119.6이 나온 것을 보아 예측 시 평균적으로 약 120명 정도의 오차가 있을 것이라는 것을 알 수 있습니다. 저 값을 줄여나가는 것이 모델의 방향이 될 것입니다.

---------------------------------------

### 7. 모델 조정 - Scaling

앞서 모았던 모델은 각 행렬의 값이 0, 1 혹은 어쩌면 2가 나올 수도 있는 메뉴판 이미지였습니다. 값의 범위가 너무 다르고 데이터 양이 많지 않기 때문에 모델이 정확한 값을 도출할 확률은 그만큼 적을 수 밖에 없습니다. 따라서 최대최소로 행렬값 자체의 범위를 줄여주는 Scaling 기법을 적용하였습니다.

In [0]:
ma1 = np.zeros((b1.shape[0],ingre_size,method_size))
ma1.shape

> 마찬가지로 0으로 구성된 행렬을 만들어줍니다

In [0]:
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [0]:
#최종 전처리 - scaling : 평균 & 표준편차 사용

for i in range(b1.shape[0]):
    print('date : ', b2['date'][i])
    print('l.men : ', b2['l.men'][i])
    cnt = 0 
    for k in range(b1.shape[1]): 
        tmp = b1[[0,1,2,3,4,5,6,7,8,9,10,11]][i:i+1]
        if tmp[k].values[0] in a:
            ma1[i] += a[tmp[k].values[0]]
            cnt += 1
            print('all_count : ', i+1)
            print('Updating..... ', tmp[k].values[0])
            print('count : ', cnt)
            print('menu : \n', tmp)
            print('array : \n', ma1[i])
            print('.')
    ma1[i] = scale(ma1[i])
    print('Final array : \n', ma1[i])
    print('.')

> 마찬가지로 날짜별 메뉴판을 만들어줍니다.

In [0]:
trainX1, testX1, trainY1, testY1 = train_test_split(ma1, y, test_size=0.3, random_state=5)

> 마찬가지로 Training, Test Set을 나눠줍니다.

In [0]:
# Store training stats
history1 = model.fit(trainX1.reshape([-1,ingre_size, method_size,1]), trainY1, 
                    epochs=EPOCHS
                    )

> 앞서 진행한 것과 같이 같은 모델로 학습합니다.

In [0]:
# 6. 모델 평가하기
loss_and_metrics1 = model.evaluate(testX1.reshape([-1,ingre_size, method_size,1]), testY1)
print('loss_and_metrics : ' + str(loss_and_metrics1))

> 마찬가지로 모델을 평가합니다. 119.4로 아까보다 0.2가 나아진 것을 볼 수 있지만 눈에 띌만한 값이 아니므로 Scaling 자체가 큰 의미가 있다고는 할 수 없을 것 같습니다.

In [0]:
model.predict(testX1.reshape([-1,ingre_size, method_size,1]))

> 마지막으로 Test Set의 값을 통해 직접 값을 예측해봅니다. 향후에 모델이 더 나아지고 이 값을 다른 값에 전달할 때 예측하는 것이 필요합니다.

<hr/>

## 8. 결론

#### : 메뉴판 이미지(행렬) 데이터를 통한 CNN 기법 적용은 오차율을 낮추는 것에 큰 효과를 보지는 못했지만(오차율 약 9~10퍼센트) 맛, Pred l.men 데이터 추가 적용, Parameter Tuning, Ensemble 기법 적용을 통해 보다 나은 모델을 만들 수 있으리라 기대합니다. 